## L1 Models Prediction Observation

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [7]:
def compute_rmse(actual, predicted):
    return np.sqrt(((actual - predicted) ** 2).mean())

def compute_mape(actual, predicted):
    return 100 * np.mean(np.abs((actual - predicted) / actual))

def compute_mae(actual, predicted):
    return np.mean(np.abs(actual - predicted))

In [8]:
# Define the data file
IS_FILTERED = True
ARCHIVED = None

SHOW_HEADER = True

# Import the data
datafile = 'prediction_result_filtered.csv' if IS_FILTERED else 'prediction_result.csv'
archived = 'archive/' + str(ARCHIVED) + '/' if ARCHIVED else ''
df = pd.read_csv('../source/l1_prediction_dataset/' + archived + datafile)
df['Time'] = pd.to_datetime(df['Time'])
df['FormattedTime'] = df['Time'].dt.strftime('%H:%M:%S')
df = df.sort_values(by='FormattedTime')

# Indicate which columns to ignore
IGNORED_COLS = ['Time', 'FormattedTime', 'Actual'] 
if IS_FILTERED: IGNORED_COLS.append('Raw')

In [9]:
# Compute RMSE and MAPE for each model
for column in df.columns:
    if column not in IGNORED_COLS:
        actual = df['Raw'] if IS_FILTERED else df['Actual']
        rmse = compute_rmse(actual, df[column])
        mape = compute_mape(actual, df[column])
        mae = compute_mae(actual, df[column])
        print(f"{column} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, MAPE: {mape:.2f}%")

ARIMA - RMSE: 0.0817, MAE: 0.0604, MAPE: 47.28%
ETS - RMSE: 0.1246, MAE: 0.0877, MAPE: 66.75%
LSTM - RMSE: 0.0573, MAE: 0.0442, MAPE: 35.68%
CNN - RMSE: 0.0580, MAE: 0.0446, MAPE: 36.06%
GRU - RMSE: 0.0556, MAE: 0.0429, MAPE: 34.67%


In [10]:
# Plot the data
fig, axes = plt.subplots(
    nrows=len(df.columns) - len(IGNORED_COLS), 
    figsize=(10, 6 * (len(df.columns) - len(IGNORED_COLS)))
    )

# Ensure axes is always a list or array
if not isinstance(axes, np.ndarray):
    axes = [axes]

ax_idx = 0  # Separate index for accessing axes
header = (str(ARCHIVED) if ARCHIVED is not None else 'Current') + " - "
header = header if SHOW_HEADER else ''

for column in df.columns:
    if column not in IGNORED_COLS:
        actual_label = 'Reduced Noise' if IS_FILTERED else 'Raw'
        actual_color = 'green' if IS_FILTERED else 'orange'
        axes[ax_idx].plot(df['FormattedTime'], df[column], label=column)
        if IS_FILTERED: axes[ax_idx].plot(df['FormattedTime'], df['Raw'], label='Raw', linestyle='-.', color='orange')  # Added this line to plot the Raw values
        axes[ax_idx].plot(df['FormattedTime'], df['Actual'], label=actual_label, linestyle='--', color=actual_color)

        # Sparse Tick Labels logic
        n = 12  # Display every n-th label 
        for index, label in enumerate(axes[ax_idx].xaxis.get_ticklabels()):
            if index % n != 0:
                label.set_visible(False)

        # Rotate the visible x-axis labels
        axes[ax_idx].tick_params(axis='x', rotation=45)

        axes[ax_idx].legend()
        # axes[ax_idx].grid(True) # Uncomment this line to show grid lines
        title = f"{header}{column} vs Raw"
        if IS_FILTERED: title += ' vs Reduced Noise'
        axes[ax_idx].set_title(title)
        ax_idx += 1  # Increment ax_idx

plt.tight_layout()
plt.show()